In [ ]:
!unzip data.zip

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob

In [3]:
class_names = [name[11:] for name in glob.glob('data/train/*')]
class_names = dict(zip(range(0,len(class_names)), class_names))
print (class_names)

def load_dataset(path, num_per_class=-1):
    data = []
    labels = []
    for id, class_name in class_names.items():
        img_path_class = glob.glob(path + class_name + '/*.jpg')
        if num_per_class > 0:
            img_path_class = img_path_class[:num_per_class]
        labels.extend([id]*len(img_path_class))
        for filename in img_path_class:
            data.append(cv2.imread(filename, 0))
    return data, labels

# load training dataset
train_data, train_label = load_dataset('data/train/', 100)
train_num = len(train_label)

# load testing dataset
test_data, test_label = load_dataset('data/test/', 100)
test_num = len(test_label)

{0: 'Office', 1: 'Coast', 2: 'Bedroom', 3: 'LivingRoom', 4: 'Forest', 5: 'Street', 6: 'Industrial', 7: 'Highway', 8: 'Mountain', 9: 'InsideCity', 10: 'Suburb', 11: 'OpenCountry', 12: 'Store', 13: 'Kitchen'}


In [ ]:
# compute dense SIFT
def computeSIFT(data):
    x = []
    for i in range(0, len(data)):
        sift = cv2.xfeatures2d.SIFT_create()
        img = data[i]
        step_size = 15
        kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, img.shape[0], step_size) for y in range(0, img.shape[1], step_size)]
        dense_feat = sift.compute(img, kp)
        x.append(dense_feat[1])

    return x

# extract dense sift features from training images
x_train = computeSIFT(train_data)
x_test = computeSIFT(test_data)

all_train_desc = []
for i in range(len(x_train)):
    for j in range(x_train[i].shape[0]):
        all_train_desc.append(x_train[i][j,:])

all_train_desc = np.array(all_train_desc)

In [ ]:
from sklearn.cluster import KMeans

def clusterFeatures(all_train_desc, k):
    kmeans = KMeans(n_clusters=k, random_state=0).fit(all_train_desc)
    return kmeans

In [ ]:
k = 200
kmeans = clusterFeatures(all_train_desc, k)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
import math

def extract_denseSIFT(img):
    DSIFT_STEP_SIZE = 2
    sift = cv2.xfeatures2d.SIFT_create()
    disft_step_size = DSIFT_STEP_SIZE
    keypoints = [cv2.KeyPoint(x, y, disft_step_size)
            for y in range(0, img.shape[0], disft_step_size)
                for x in range(0, img.shape[1], disft_step_size)]

    descriptors = sift.compute(img, keypoints)[1]
    return descriptors


# form histogram with Spatial Pyramid Matching upto level L with codebook kmeans and k codewords
def getImageFeaturesSPM(L, img, kmeans, k):
    W = img.shape[1]
    H = img.shape[0]
    h = []
    for l in range(L+1):
        w_step = math.floor(W/(2**l))
        h_step = math.floor(H/(2**l))
        x, y = 0, 0
        for i in range(1,2**l + 1):
            x = 0
            for j in range(1, 2**l + 1):
                desc = extract_denseSIFT(img[y:y+h_step, x:x+w_step])
                #print("type:",desc is None, "x:",x,"y:",y, "desc_size:",desc is None)
                predict = kmeans.predict(desc)
                histo = np.bincount(predict, minlength=k).reshape(1,-1).ravel()
                weight = 2**(l-L)
                h.append(weight*histo)
                x = x + w_step
            y = y + h_step

    hist = np.array(h).ravel()
    # normalize hist
    dev = np.std(hist)
    hist -= np.mean(hist)
    hist /= dev
    return hist


# get histogram representation for training/testing data
def getHistogramSPM(L, data, kmeans, k):
    x = []
    for i in range(len(data)):
        hist = getImageFeaturesSPM(L, data[i], kmeans, k)
        x.append(hist)
    return np.array(x)

In [ ]:
train_histo = getHistogramSPM(2, train_data, kmeans, k)
test_histo = getHistogramSPM(2, test_data, kmeans, k)

In [ ]:
from sklearn.svm import LinearSVC

for c in np.arange(0.000307, 0.001, 0.0000462):
    clf = LinearSVC(random_state=0, C=c)
    clf.fit(train_histo, train_label)
    predict = clf.predict(test_histo)
    print ("C =", c, ",\t\t Accuracy:", np.mean(predict == test_label)*100, "%")

C = 0.000307 ,		 Accuracy: 69.85714285714286 %
C = 0.00035319999999999997 ,		 Accuracy: 69.64285714285714 %
C = 0.00039939999999999995 ,		 Accuracy: 69.71428571428572 %
C = 0.00044559999999999993 ,		 Accuracy: 70.07142857142857 %
C = 0.0004917999999999999 ,		 Accuracy: 70.07142857142857 %
C = 0.0005379999999999998 ,		 Accuracy: 70.14285714285714 %
C = 0.0005841999999999999 ,		 Accuracy: 70.0 %
C = 0.0006303999999999999 ,		 Accuracy: 69.78571428571428 %
C = 0.0006765999999999999 ,		 Accuracy: 69.92857142857143 %
C = 0.0007227999999999998 ,		 Accuracy: 70.0 %
C = 0.0007689999999999998 ,		 Accuracy: 70.0 %
C = 0.0008151999999999999 ,		 Accuracy: 70.07142857142857 %
C = 0.0008613999999999998 ,		 Accuracy: 69.92857142857143 %
C = 0.0009075999999999997 ,		 Accuracy: 69.78571428571428 %
C = 0.0009537999999999998 ,		 Accuracy: 69.71428571428572 %
C = 0.0009999999999999998 ,		 Accuracy: 69.57142857142857 %
